In [3]:
import dask.dataframe as dd
import pandas as pd
df_resp = dd.read_csv('/project2/wparker/SIPA_data/RCLIF_respiratory_support.csv')
df_resp.head()

,Unnamed: 0,C19_HAR_ID,recorded_time,device_name,lpm,fio2,peep,prone
0,1,91,2020-09-29 20:22:00,Other,0.0,NaN,NaN,0.0
1,2,91,2020-09-30 04:13:00,Other,NaN,NaN,NaN,0.0
2,3,91,2020-09-30 08:00:00,Other,0.0,NaN,NaN,0.0
3,4,91,2020-09-30 12:00:00,Other,0.0,NaN,NaN,NaN
4,5,91,2020-09-30 15:55:00,Other,0.0,NaN,NaN,NaN


In [71]:
#get just invasive or non-invasive mechanical ventilation
vent = df_resp.loc[(df_resp['device_name']=='Vent') | (df_resp['device_name']=='NIPPV')]
# minimum time by person
vent = vent.compute()
vent['recorded_time'] = pd.to_datetime(vent['recorded_time'], format='%Y-%m-%d %H:%M:%S')

vent_grouped = vent.groupby('C19_HAR_ID')['recorded_time'].agg(['min'])
vent_grouped = vent_grouped.rename(columns={'min': 'min_resp_support_time'})
vent_grouped.head()


,min_resp_support_time
C19_HAR_ID,
1902,2023-03-03 05:30:00
2750,2023-02-14 20:18:00
4856,2021-03-12 08:36:00
6984,2023-01-22 03:40:00
9041,2021-03-24 16:33:00


In [7]:
df_meds = dd.read_csv('/project2/wparker/SIPA_data/RCLIF_meds_admin_conti.csv')
df_meds.head()

,Unnamed: 0,C19_HAR_ID,admin_time,medication_name,med_dose,med_dose_unit,med_name
0,1,23065362,2022-10-06 17:41:00,PHENYLEPHRINE 50 MG/250 ML (200 MCG/ML) IN 0.9...,0.5,mcg/kg/min,phenylephrine
1,2,23065362,2022-10-06 19:27:00,PHENYLEPHRINE 50 MG/250 ML (200 MCG/ML) IN 0.9...,0.7,mcg/kg/min,phenylephrine
2,3,23065362,2022-10-06 19:34:00,PHENYLEPHRINE 50 MG/250 ML (200 MCG/ML) IN 0.9...,0.5,mcg/kg/min,phenylephrine
3,4,23065362,2022-10-06 18:17:00,PHENYLEPHRINE 50 MG/250 ML (200 MCG/ML) IN 0.9...,1.5,mcg/kg/min,phenylephrine
4,5,23065362,2022-10-06 08:19:00,PHENYLEPHRINE 50 MG/250 ML (200 MCG/ML) IN 0.9...,0.5,mcg/kg/min,phenylephrine


In [70]:
pressors = df_meds.loc[(df_meds['med_name']=='phenylephrine') | 
                       (df_meds['med_name']=='epinephrine') | 
                       (df_meds['med_name']=='vasopressin') | 
                       (df_meds['med_name']=='dopamine') |
                       (df_meds['med_name']=='dobutamine') |
                       (df_meds['med_name']=='norepinephrine') |
                       (df_meds['med_name']=='angiotensin') |
                       (df_meds['med_name']=='isoproterenol')]
pressors = pressors[["C19_HAR_ID", "admin_time"]]
#print(len(pressors))
pressors_no_na = pressors.dropna()
#print(len(pressors_no_na))
pressors_no_na = pressors_no_na.compute()

pressors_no_na['admin_time'] = pd.to_datetime(pressors_no_na['admin_time'], format='%Y-%m-%d %H:%M:%S')

pressors_grouped = pressors_no_na.groupby('C19_HAR_ID')['admin_time'].agg(['min'])
pressors_grouped = pressors_grouped.rename(columns={'min': 'min_pressor_time'})
pressors_grouped.head()

,min_pressor_time
C19_HAR_ID,
704,2021-06-27 07:20:00
2750,2023-02-17 09:48:00
6100,2023-06-30 04:07:00
10593,2020-06-04 02:52:00
10932,2021-12-18 21:43:00


In [72]:
df = pressors_grouped.join(vent_grouped, on='C19_HAR_ID', how='outer')
#print(len(pressors_grouped))
#print(len(vent_grouped))
#print(len(df))

df.head()

,C19_HAR_ID,min_pressor_time,min_resp_support_time
704.0,704,2021-06-27 07:20:00,NaT
2750.0,2750,2023-02-17 09:48:00,2023-02-14 20:18:00
6100.0,6100,2023-06-30 04:07:00,NaT
10593.0,10593,2020-06-04 02:52:00,2020-06-05 00:46:00
10932.0,10932,2021-12-18 21:43:00,2021-12-19 10:03:00


In [74]:
df = df[['C19_HAR_ID','min_pressor_time','min_resp_support_time']]


df['life_support_start'] = df[['min_pressor_time','min_resp_support_time']].min(axis=1)
df.head()

,C19_HAR_ID,min_pressor_time,min_resp_support_time,life_support_start
704.0,704,2021-06-27 07:20:00,NaT,2021-06-27 07:20:00
2750.0,2750,2023-02-17 09:48:00,2023-02-14 20:18:00,2023-02-14 20:18:00
6100.0,6100,2023-06-30 04:07:00,NaT,2023-06-30 04:07:00
10593.0,10593,2020-06-04 02:52:00,2020-06-05 00:46:00,2020-06-04 02:52:00
10932.0,10932,2021-12-18 21:43:00,2021-12-19 10:03:00,2021-12-18 21:43:00


In [75]:
df = df[['C19_HAR_ID', 'life_support_start']]
print(df)

         C19_HAR_ID  life_support_start
704.0           704 2021-06-27 07:20:00
2750.0         2750 2023-02-14 20:18:00
6100.0         6100 2023-06-30 04:07:00
10593.0       10593 2020-06-04 02:52:00
10932.0       10932 2021-12-18 21:43:00
...             ...                 ...
NaN        29997532 2020-09-01 04:27:00
NaN        29998146 2021-10-16 01:37:00
NaN        29999119 2022-01-08 08:46:00
NaN        29999172 2021-02-01 13:27:00
NaN        29999273 2021-11-22 22:29:00

[19864 rows x 2 columns]
